<a href="https://www.kaggle.com/code/mikedelong/python-eda-mostly-bars-and-scatters?scriptVersionId=143034783" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
from numpy import nan
import pandas as pd

df = pd.read_csv(filepath_or_buffer='/kaggle/input/commonlit-texts/commonlit_texts.csv')
df['description_length'] = df['description'].apply(func=lambda x: len(x.split()))
df.head()

In [ ]:
df.info()

In [ ]:
df.nunique()

In [ ]:
from plotly.express import histogram
histogram(data_frame=df, x='grade')

In [ ]:
from plotly.express import bar
bar(data_frame=df['author'].value_counts().nlargest(n=20).to_frame().reset_index(), x='author', y='count')

Weird how the top twenty is dominated by authors who are not exactly household names.

In [ ]:
bar(data_frame=df, x='genre', category_orders= {'genre': sorted(df['genre'].unique().tolist())})

Maybe a volumetric plot would be better. Let's try.

In [ ]:
from plotly.express import treemap
treemap(data_frame=df[['genre', 'grade']].groupby(by=['genre', 'grade']).size().reset_index().rename(columns={0: 'count'}), names='genre',  values='count', path=['genre'])

In [ ]:
treemap(data_frame=df[['genre', 'grade']].groupby(by=['genre', 'grade']).size().reset_index().rename(columns={0: 'count'}), names='grade', values='count', path=['grade'])

In [ ]:
treemap(data_frame=df[['author', 'genre']].groupby(by=['author', 'genre']).size().reset_index(),
        names='author', values=0, path=['author'], color='genre',)

The corpus is mostly low-cardinality authors.

In [ ]:
from plotly.express import scatter
scatter(data_frame=df, x='grade', y='lexile', color='genre', hover_name='title', trendline='ols', trendline_scope='overall')

Lexile is a measure of reading difficulty; we expect it to rise linearly with grade.

In [ ]:
from plotly.express import scatter_matrix
scatter_matrix(data_frame=df, dimensions=['grade', 'lexile', 'description_length'], color='genre', hover_name='title')

These are more fun than informative. Look at all the pretty colors.

In [ ]:
from plotly.express import violin
violin(data_frame=df, x='grade', y='lexile', hover_name='title')

This graph looks weird, but it shows how the bulk of the lexile distribution rises slowly with the grade. Maybe a ridge/joy plot would be helpful here.

In [ ]:
scatter(data_frame=df, x='grade', y='description_length', color='genre', hover_name='title', trendline='ols', trendline_scope='overall')

We would like description length to be a proxy for something, but it doesn't seem to be. Desscriptions get longer for texts for higher grades, but not much.

In [ ]:
scatter(data_frame=df, y='lexile', x='description_length', color='genre', hover_name='title', trendline='ols', trendline_scope='overall')

Description length and lexile score are positively correlated for the corpus overall.

In [ ]:
from wordcloud import WordCloud
from matplotlib.pyplot import axis
from matplotlib.pyplot import imshow
from matplotlib.pyplot import subplots
subplots(figsize=(12, 12))
axis('off')
imshow(WordCloud().generate(' '.join(df['description'].values.tolist())))

Even after we remove common stopwords the descriptions are dominated by filler words. Words like discuss, describe, speaker, poem, etc. are more form than content in a corpus like this.